# CPU & NIC Power Modeling Interface

This notebook provides a framework for visualizing and managing CPU and NIC power models.

It is designed to support analysis of server hardware configurations and their power consumption characteristics.



## Purpose

The notebook is designed to:

- Load power models for CPUs from saved JSON files.
- Retrieve and display NIC power consumption data.
- Provide interactive widgets and visualizations using Matplotlib and ipywidgets.
- Support model lookup, comparison, and visual analysis.



## Directory and File Structure

- **Model Input Directory:** data/cpu_models/
- **Model File Naming Convention:** cpu_model_<server_name>.json
- **NIC Power Data:** Defined within the notebook in the NIC_POWER_DATABASE dictionary.

Each model file should be a valid JSON file containing expected keys like node_name and other CPU-related information.

Use these visualizations to compare power usage across servers, view NIC impact, and analyze trends.

## Dependencies

For dependencies take a look at the **README.md** file.

## NIC Power Reference

The NIC power data is currently defined directly within the notebook using a hardcoded dictionary (NIC_POWER_DATABASE). This serves as a temporary reference and is **not yet integrated with a full NIC model database**.

The plan is to eventually support:
- Loading NIC power models from structured files (similar to CPU models).
- Managing NIC configuration dynamically through the interface.
- Associating NICs with servers in a more scalable, modular way.

Current hardcoded values:

| NIC Model                 | Power (W) |
|---------------------------|-----------|
| Intel 10-Gigabit X540-AT2 | 7.5       |
| 82579LM Gigabit           | 4.0       |
| Intel E810-XXV            | 12.5      |


This placeholder approach allows for basic NIC power accounting in the total power estimation, but it will be replaced with a proper data model in future development.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.colors as mcolors
import os
import json
from ipywidgets import ToggleButton, VBox, HBox, IntSlider, Label, Output, ToggleButtons, Button, HTML
from IPython.display import display, clear_output
from typing import Dict

# ``` Constants ```
CPU_COLOR = "#17a2b8"  # Bootstrap "info" blue
NIC_COLOR = "#28a745"  # Bootstrap "success" green
CPU_MODEL_FOLDER = "data/cpu_models"
MODEL_OUTPUT = "cpu_model"

# ``` NIC Power Database ```
NIC_POWER_DATABASE = {
    "Intel 10-Gigabit X540-AT2": 7.5,
    "82579LM Gigabit": 4.0,
    "Intel E810-XXV": 12.5
}

def load_cpu_model() -> Dict:
    server_name = input("Server Name: ")
    name = f"{MODEL_OUTPUT}_{server_name}.json"
    path = os.path.join(CPU_MODEL_FOLDER, name)

    if not os.path.exists(path):
        raise FileNotFoundError(f"Could not find cpu_model file {path}")
    with open(path, "r") as file:
        model = json.load(file)
        print(f"Loaded model from {path}")
        return model

def load_all_cpu_models() -> Dict[str, Dict]:
    models = {}
    for filename in os.listdir(CPU_MODEL_FOLDER):
        if filename.startswith(MODEL_OUTPUT) and filename.endswith(".json"):
            path = os.path.join(CPU_MODEL_FOLDER, filename)
            with open(path, "r") as file:
                model = json.load(file)
                node_name = model.get("node_name", filename)
                models[node_name] = model
    if not models:
        raise FileNotFoundError(f"No CPU model files found in {CPU_MODEL_FOLDER}.")
    print(f"Loaded {len(models)} models.")
    return models

def nic_power_lookup(model_name: str) -> float:
    return NIC_POWER_DATABASE.get(model_name, 0.0)

def adjust_color(base: str, factor: float) -> str:
    rgb = mcolors.to_rgb(base)
    adjusted = tuple(min(1, c * factor) for c in rgb)
    return mcolors.to_hex(adjusted)

def section_header(title: str, size: int = 20) -> HTML:
    return HTML(f"""
        <div style="
            font-weight: bold;
            font-size: {size}px;
            padding: 10px 0;
            margin: 20px 0 10px 0;
            border-bottom: 2px solid #999;">
            {title}
        </div>
    """)

In [ ]:
# load models
models = load_all_cpu_models()

# User Guide

Explains the usage of the prediction plots.

## Global Settings (Affects All Nodes)

### Load Level Toggles

You can toggle between:
- Idle (0%)
- 25% Load
- 50% Load
- 75% Load
- 100% Load

These global load levels are used when plotting in separate mode.

Additional buttons:
- Activate All Loads
- Deactivate All Loads

If using combined plot mode, per-node load selection overrides global settings.

### Prediction Method

Choose how CPU power is estimated:
- Linear: Uses a basic linear formula.
- Polynomial: Uses a higher-order equation if defined in the model.

### Plot Mode

Choose how to display results:
- Separate Plots: One plot per node, showing all selected load levels.
- Combined Plot: One stacked bar showing the combined power usage across all selected nodes.

## Node Selection and Configuration

Use the toggle buttons to select one or more nodes.

Once selected, a configuration panel appears for each node with:

### Per-Node CPU Load (for Combined Plot Only)

Select one CPU load level for that node.
Ignored in separate plot mode, which uses the global load levels instead.

### NIC Selection

Toggle on/off available NICs for each node. These affect NIC power added to the chart.

Additional buttons:
- Activate All NICs
- Deactivate All NICs

> NIC values come from a hardcoded dictionary in the notebook. A structured NIC model system is planned for the future.

### Core Slider

Adjust how many CPU cores are active for that node. The selected load level is applied as a percentage across this number of cores.

## Plot Types and Interpretation

### Separate Plots

Each selected node gets its own chart.

- X-axis: CPU load levels
- Y-axis: Predicted power (watts)
- Each bar:
  - Bottom portion = CPU power
  - Top portion = NIC power
- Total combined power is labeled above each bar

Use this to compare how a single node behaves under different workloads.

### Combined Plot

One chart displays a single stacked bar for all selected nodes.

Each node contributes:
- A CPU segment (color adjusted for node index)
- A NIC segment

The full system power is shown at the top.

Use this to estimate total experiment consumption.

In [ ]:
def interactive_multi_node(models: Dict[str, Dict]) -> None:
    # ``` Global Section ```
    load_options = ["Idle (0%)", "25% Load", "50% Load", "75% Load", "100% Load"]

    global_load_buttons = {}
    for name in load_options:
        btn = ToggleButton(value=True, description=name, layout={'width': '120px'})
        btn.button_style = 'info' if btn.value else ''
        btn.observe(lambda change, b=btn: (update_global_load_button_style(b), plot()), names='value')
        global_load_buttons[name] = btn

    def update_global_load_button_style(btn):
        btn.button_style = 'info' if btn.value else ''

    method_toggle = ToggleButtons(
        options=[('Linear', 'linear'), ('Polynomial', 'polynomial')],
        value='linear'
    )
    method_toggle.observe(lambda c: plot(), names='value')

    plot_mode_toggle = ToggleButtons(
        options=[('Separate Plots', 'separate'), ('Combined Plot', 'combined')],
        value='separate'
    )
    plot_mode_toggle.observe(lambda c: plot(), names='value')

    def toggle_all(buttons, value, style_func):
        for b in buttons.values():
            b.value = value
            style_func(b)

    activate_loads = Button(description="Activate All Loads", layout={'width': '160px'})
    deactivate_loads = Button(description="Deactivate All Loads", layout={'width': '160px'})
    activate_loads.on_click(lambda _: toggle_all(global_load_buttons, True, update_global_load_button_style))
    deactivate_loads.on_click(lambda _: toggle_all(global_load_buttons, False, update_global_load_button_style))

    # ``` Node Selection Section ```
    node_buttons = {node: ToggleButton(value=False, description=node, layout={'width': '120px'}) for node in models}

    node_settings = {}
    node_configs_output = Output()
    plot_output = Output()

    def update_node_button_style(btn):
        btn.button_style = 'warning' if btn.value else ''

    def build_node_config(node_name: str):
        model = models[node_name]
        num_cores = model.get("cpu_info", {}).get("cores", 4)

        load_buttons = {}
        def on_load_change(change, name):
            if change['new']:
                for other_name, other_btn in load_buttons.items():
                    if other_name != name:
                        other_btn.unobserve_all()
                        other_btn.value = False
                        other_btn.button_style = ''
                        other_btn.observe(lambda c, n=other_name: on_load_change(c, n), names='value')
                    else:
                        other_btn.button_style = 'info'
                plot()

        for name in load_options:
            btn = ToggleButton(value=False, description=name, layout={'width': '120px'})
            btn.observe(lambda change, n=name: on_load_change(change, n), names='value')
            load_buttons[name] = btn

        # Default select Idle
        default = "Idle (0%)"
        load_buttons[default].unobserve_all()
        load_buttons[default].value = True
        load_buttons[default].button_style = 'info'
        load_buttons[default].observe(lambda c, n=default: on_load_change(c, n), names='value')

        nic_buttons = {
            nic: ToggleButton(value=True, description=nic, layout={'width': '140px'})
            for nic in NIC_POWER_DATABASE
        }

        def on_nic_change(change, b):
            b.button_style = 'success' if b.value else ''
            plot()

        for b in nic_buttons.values():
            b.button_style = 'success' if b.value else ''
            b.observe(lambda change, b=b: on_nic_change(change, b), names='value')

        core_slider = IntSlider(value=num_cores, min=1, max=num_cores, step=1, description="Cores:")
        core_slider.observe(lambda c: plot(), names='value')

        activate_nics = Button(description="Activate All NICs", layout={'width': '120px'})
        deactivate_nics = Button(description="Deactivate All NICs", layout={'width': '120px'})
        activate_nics.on_click(lambda _: toggle_all(nic_buttons, True, lambda b: setattr(b, 'button_style', 'success')))
        deactivate_nics.on_click(lambda _: toggle_all(nic_buttons, False, lambda b: setattr(b, 'button_style', '')))

        node_settings[node_name] = {
            'model': model,
            'load_buttons': load_buttons,
            'nic_buttons': nic_buttons,
            'core_slider': core_slider
        }

        capitalized = node_name.capitalize()
        return VBox([
            section_header(f"{capitalized} Configuration", size=16),
            VBox([
                Label("Select CPU Load Level", style={'font_weight': 'bold'}),
                HBox(list(load_buttons.values()), layout={'flex_wrap': 'wrap'})
            ]),
            VBox([
                Label("Select NICs", style={'font_weight': 'bold'}),
                HBox(list(nic_buttons.values()), layout={'flex_wrap': 'wrap'}),
                HBox([activate_nics, deactivate_nics], layout={'gap': '8px'})
            ]),
            Label("Select Number of Cores", style={'font_weight': 'bold'}),
            core_slider
        ], layout={'margin': '25px 0 0 0'})

    def update_node_configs(change=None):
        with node_configs_output:
            clear_output()
            active_nodes = [n for n, b in node_buttons.items() if b.value]
            display(VBox([build_node_config(n) for n in active_nodes]))
            plot()

    for node, btn in node_buttons.items():
        update_node_button_style(btn)
        btn.observe(lambda change, b=btn: (update_node_button_style(b), update_node_configs()), names='value')

    # ``` Plot Function ```
    def plot():
        with plot_output:
            clear_output()
            active_nodes = [n for n, b in node_buttons.items() if b.value]
            if not active_nodes:
                print("Select at least one node.")
                return

            selected_loads = [name for name, b in global_load_buttons.items() if b.value]
            if not selected_loads:
                print("Select at least one global load level.")
                return

            if plot_mode_toggle.value == 'separate':
                for idx, node in enumerate(active_nodes):
                    plot_node_separate(node, idx)
            else:
                plot_nodes_combined(active_nodes)

    def plot_node_separate(node, idx):
        s = node_settings[node]
        model = s['model']
        cores = s['core_slider']
        nic_buttons = s['nic_buttons']

        selected_loads = [name for name, btn in global_load_buttons.items() if btn.value]
        if not selected_loads:
            return

        load_map = {
            "Idle (0%)": 0.0,
            "25% Load": 0.25,
            "50% Load": 0.5,
            "75% Load": 0.75,
            "100% Load": 1.0
        }
        method = method_toggle.value
        nic_total = sum(nic_power_lookup(n) for n, b in nic_buttons.items() if b.value)

        labels = []
        cpu_powers = []
        nic_powers = []

        for load_name in selected_loads:
            load_value = load_map[load_name]
            active_cores = cores.value * load_value
            cpu_power = model["linear_model"]["p_base"] if active_cores == 0 else predict_power(model, active_cores, method)

            labels.append(load_name)
            cpu_powers.append(cpu_power)
            nic_powers.append(nic_total)

        fig, ax = plt.subplots(figsize=(8, 6))
        bar_positions = range(len(labels))
        ax.bar(bar_positions, cpu_powers, color=CPU_COLOR, edgecolor='black', label='CPU Power')
        ax.bar(bar_positions, nic_powers, bottom=cpu_powers, color=NIC_COLOR, edgecolor='black', label='NIC Power')

        for i in bar_positions:
            total = cpu_powers[i] + nic_powers[i]
            ax.text(i, total + 1, f"{total:.1f} W", ha='center', fontsize=10)

        ax.set_xticks(bar_positions)
        ax.set_xticklabels(labels)
        ax.set_ylabel("Power (W)")
        ax.set_title(f"Power Prediction for {node.capitalize()}")
        ax.legend()
        ax.grid(axis="y", linestyle="--", alpha=0.6)
        fig.tight_layout()
        plt.show()

    def plot_nodes_combined(active_nodes):
        fig, ax = plt.subplots(figsize=(6, 6))
        bottoms = [0]
        legend_handles = []

        for idx, node in enumerate(active_nodes):
            s = node_settings[node]
            model = s['model']
            cores = s['core_slider']
            load_buttons = s['load_buttons']
            nic_buttons = s['nic_buttons']

            selected_load = None
            for name, btn in load_buttons.items():
                if btn.value:
                    selected_load = name
                    break

            if selected_load is None:
                continue

            load_map = {
                "Idle (0%)": 0.0,
                "25% Load": 0.25,
                "50% Load": 0.5,
                "75% Load": 0.75,
                "100% Load": 1.0
            }
            load_value = load_map[selected_load]
            method = method_toggle.value

            nic_total = sum(nic_power_lookup(n) for n, b in nic_buttons.items() if b.value)
            active_cores = cores.value * load_value
            cpu_power = model["linear_model"]["p_base"] if active_cores == 0 else predict_power(model, active_cores, method)

            factor = 1 - idx * 0.1
            cpu_color = adjust_color(CPU_COLOR, factor)
            nic_color = adjust_color(NIC_COLOR, factor)

            legend_handles.append(mpatches.Patch(color=cpu_color, label=f"CPU ({node})"))
            legend_handles.append(mpatches.Patch(color=nic_color, label=f"NIC ({node})"))

            ax.bar(0, cpu_power, bottom=bottoms[0], color=cpu_color, edgecolor='black')
            bottoms[0] += cpu_power

            ax.bar(0, nic_total, bottom=bottoms[0], color=nic_color, edgecolor='black')
            bottoms[0] += nic_total

        total = bottoms[0]
        ax.text(0, total + 3, f"{total:.1f} W", ha='center', fontsize=12)

        ax.set_xlim(-0.5, 0.5)
        ax.set_xticks([0])
        ax.set_xticklabels(["System Configuration"])
        ax.set_ylim(0, total * 1.2)
        ax.set_ylabel("Power (W)")
        ax.set_title("Total System Power Prediction")
        ax.grid(axis="y", linestyle="--", alpha=0.6)
        ax.legend(handles=legend_handles, loc='upper left')
        fig.tight_layout()
        plt.show()

    display(VBox([
        section_header("Global Settings"),
        VBox([
            Label("Select Load Levels (Global)", style={'font_weight': 'bold'}),
            HBox(list(global_load_buttons.values()), layout={'flex_wrap': 'wrap'}),
            HBox([activate_loads, deactivate_loads], layout={'gap': '8px'}),
            Label("Select Prediction Method", style={'font_weight': 'bold'}),
            method_toggle,
            Label("Plot Mode", style={'font_weight': 'bold'}),
            plot_mode_toggle
        ]),
        section_header("Node Selection"),
        HBox(list(node_buttons.values())),
        node_configs_output,
        plot_output
    ]))


In [ ]:
interactive_multi_node(models)